In [ ]:
import pandas as pd
from artifacts.ai_pc import main_proc_change
from config.params import ProcChangeConfig
import csv
from datetime import datetime


In [ ]:
df = pd.read_csv('AAM_MEXICO_DATA_ACCELERATION_EMA2252M10022.csv')

In [ ]:
# df['READ_TIME'] = pd.to_datetime(df['READ_TIME'])
# batch_size = 8  
# df['batch'] = (df['READ_TIME'] - df['READ_TIME'].iloc[0]).dt.total_seconds() // (batch_size * 3600)

# grouped = df.groupby('batch')

# batch_results = []
# for _, group in grouped:
#     batch_start_hour = group['READ_TIME'].iloc[0].strftime('%Y%m%d%H')  # Format startHour as needed
    
#     batch_fetch_result = {
#         "data": {
#             "moldId": int(group['ID'].unique()[0]),
#             "counterId": str(group['COUNTER_ID'].unique()[0]),
#             "startHour": batch_start_hour,
#             "contractedCycleTime": 360,
#             "accelerations": group['ACCELERATIONS'].tolist(),
#             "dataAccId": group['ID'].tolist(),
#             "measurementDate": group['MEASUREMENT_DATE'].astype(str).tolist(),
#             "procChanged": [None] * len(group),
#             "similarityMetric": [None] * len(group),
#             "similarityMetricHr": [None] * len(group),
#         }
#     }
#     batch_results.append(batch_fetch_result)


In [ ]:
# Creating Batch Using MEASUREMENT_DATE(1)
# Define a function to convert datetime string to desired format
def convert_to_custom_format(datetime_str):
    datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
    return datetime_obj.strftime('%Y%m%d%H%M%S')

In [ ]:
# Fetching the Data in 8 hour Period Based as similar to what described in the fetching schema
# Set the timestamp as the DataFrame index
df['MEASUREMENT_DATE'] = pd.to_datetime(df['MEASUREMENT_DATE'],format ='%Y%m%d%H%M%S')
df.set_index('MEASUREMENT_DATE',inplace=True)
# filename = filePath.split("/")[-1].split(".")[0]
# id = filename
# Resample the data on an hourly basis and store only the non-empty hourly frames in a list
hourly_frames = []
hourlyshotsLabel = []
hourname = []
for name, group in df.resample("H"):
    if not group.empty:
        hourname.append(name)
        hourly_frames.append(group)
first_batch = hourly_frames[1:9]
df_first_batch = pd.concat(first_batch)

df.reset_index(drop = False,inplace=True)
df_first_batch.reset_index(drop = False,inplace=True)

df_first_batch['MEASUREMENT_DATE'] = df_first_batch['MEASUREMENT_DATE'].astype(str)
df_first_batch
# Apply the function to the entire column
df_first_batch['MEASUREMENT_DATE'] = df_first_batch['MEASUREMENT_DATE'].apply(convert_to_custom_format)
df = df_first_batch
# print(df.head())
# logger.info("DataFrame:\n%s", df)
# Convert DataFrame to the "fetchScheme Result" format
# fetch_scheme_result = df.to_dict(orient='records')

# logger.info('fetchScheme Result : %s', fetch_scheme_result)
# Print the result
# print()
# for row in fetch_scheme_result:
#     # print(row)
#     pass

In [ ]:
df

In [ ]:
# Creating Batch Using MEASUREMENT_DATE(3)
try: 
    dummy_fetchResult = {
        "data": {
            "moldId": int(df['MOLD_ID'].unique()[0]),
            "counterId": str(df['COUNTER_ID'].unique()[0]),
            "startHour":str(df['MEASUREMENT_DATE'][0])[:-4], # Left side of the Inference Boundary
            "contractedCycleTime": 360,
            "accelerations": df['ACCELERATIONS'].tolist(),
            "dataAccId": df['ID'].tolist(),
            "measurementDate": df['MEASUREMENT_DATE'].tolist(),
            "procChanged": [None] * len(df['ID'].tolist()),  # Example process change indicators
            "similarityMetric": [None] * len(df['ID'].tolist()),
            "similarityMetricHr": [None] * len(df['ID'].tolist()),
        }
    }
except Exception as e:
    print(e)
    dummy_fetchResult = {
    "data": {
        "moldId": df['TERMINAL_ID'].unique()[0],
        "counterId": str(df['COUNTER_ID'].unique()[0]),
        "startHour":str(df['MEASUREMENT_DATE'][0])[:-4], # Left side of the Inference Boundary
        "contractedCycleTime": 360,
        "accelerations": df['ACCELERATIONS'].tolist(),
        "dataAccId": df['ID'].tolist(),
        "measurementDate": df['MEASUREMENT_DATE'].tolist(),
        "procChanged": [None] * len(df['ID'].tolist()),  # Example process change indicators
        "similarityMetric": [None] * len(df['ID'].tolist()),
        "similarityMetricHr": [None] * len(df['ID'].tolist()),
    }
    }

In [ ]:
result_df = pd.DataFrame(dummy_fetchResult)
result_df

In [ ]:
from artifacts.utils import generate_hrs_template
print('Batch Start Hour: ',str(df['MEASUREMENT_DATE'][0])[:-4])
print(generate_hrs_template(
    str(df['MEASUREMENT_DATE'][0])[:-4],
    4,
    2,
))
batch_start_hour=str(df['MEASUREMENT_DATE'][0])[:-4]

In [ ]:
# Simulate no data available for any timestamp
no_data_batch = pd.DataFrame(columns=df_first_batch.columns)
# Now you can use this no_data_batch to generate dummy_fetchResult
df = no_data_batch

In [ ]:
# Use irregular timestamps for the batch
irregular_timestamps_batch = df_first_batch.copy()
irregular_timestamps_batch = irregular_timestamps_batch.iloc[::2]  # Select every other timestamp
# Now you can use this irregular_timestamps_batch to generate dummy_fetchResult
df= irregular_timestamps_batch

In [ ]:
# Simulate missing data for some timestamps
partial_data_batch = df_first_batch.copy()
partial_data_batch = partial_data_batch[~partial_data_batch['MEASUREMENT_DATE'].isin(['2023-04-26 16:00:00', '2023-04-26 18:00:00'])]
# Now you can use this partial_data_batch to generate dummy_fetchResult
df=partial_data_batch

In [ ]:
from config.params import ProcChangeConfig

result=main_proc_change(dummy_fetchResult,
                THRESHOLD_SIM_METRIC=ProcChangeConfig.THRESHOLD_SIM_METRIC,
                N_PREV_RECS=ProcChangeConfig.N_PREV_RECS,
                FIRST_SECTION_PROP=ProcChangeConfig.FIRST_SECTION_PROP,
                BIN_WIDTH_FIRST_SECTION=ProcChangeConfig.BIN_WIDTH_FIRST_SECTION,
                BIN_WIDTH_SECOND_SECTION=ProcChangeConfig.BIN_WIDTH_SECOND_SECTION,

                )
print(result)
# Convert the result to a DataFrame
result_df = pd.DataFrame(result)
result_df
# # Save the DataFrame to a CSV file
# result_df.to_csv('PCD_MAIN_PROCESS_CHANGE_MEASUREMENT_DATE.csv', index=False)

In [ ]:
from artifacts.ai_pc import identify_process_change
from artifacts.ai_pc import get_hourly_summary
from artifacts.ai_pc import parse_individual_records
records_all = parse_individual_records(
        dummy_fetchResult,
        FIRST_SECTION_PROP=ProcChangeConfig.FIRST_SECTION_PROP,
        BIN_WIDTH_FIRST_SECTION=ProcChangeConfig.BIN_WIDTH_FIRST_SECTION,
        BIN_WIDTH_SECOND_SECTION=ProcChangeConfig.BIN_WIDTH_SECOND_SECTION,
)
records_all

# df=pd.DataFrame(records_all)
# df

# custom_width = 130
# pd.set_option('display.max_colwidth', custom_width)

# # Display the DataFrame
# df

ach=get_hourly_summary(
    batch_start_hour,
    records_all,
)
ach

# identify_process_change(
#         ach,
#         0.7,  
#         )

In [ ]:
from artifacts.utils import generate_hrs_template
print('Batch Start Hour: ',str(df['MEASUREMENT_DATE'][0])[:-4])
print(generate_hrs_template(
    str(df['MEASUREMENT_DATE'][0])[:-4],
    4,
    2,
))


In [ ]:
batch_start_hour= str(df['MEASUREMENT_DATE'][0])[:-4]
from artifacts.utils import subtract_hours
print(subtract_hours(
            start_time=batch_start_hour,
            hrs_to_sub=2,
            input_fmt="%Y%m%d%H",
            output_fmt="%Y%m%d%H",
        ))

In [ ]:
from artifacts.ai_pc import get_hourly_summary
get_hourly_summary(
        batch_start_hour,
        records_all)

In [ ]:
from pandas import Timestamp

# Define the timestamps to keep
timestamps_to_keep = [
    Timestamp('2023-04-26 13:00:00'),
    Timestamp('2023-04-26 14:00:00'),
    Timestamp('2023-04-26 15:00:00'),
    Timestamp('2023-04-26 16:00:00'),
    Timestamp('2023-04-26 17:00:00'),
    Timestamp('2023-04-26 18:00:00'),
    Timestamp('2023-04-26 19:00:00'),
    Timestamp('2023-04-26 20:00:00')
]

# Filter the data based on the desired timestamps
filtered_data = [
    {
        "accelerations": a,
        "dataAccId": da,
        "measurementDate": md,
        "procChanged": pc,
        "similarityMetric": sm,
        "similarityMetricHr": smh
    }
    for a, da, md, pc, sm, smh in zip(
        dummy_fetchResult['data']['accelerations'],
        dummy_fetchResult['data']['dataAccId'],
        dummy_fetchResult['data']['measurementDate'],
        dummy_fetchResult['data']['procChanged'],
        dummy_fetchResult['data']['similarityMetric'],
        dummy_fetchResult['data']['similarityMetricHr']
    )
    if md in timestamps_to_keep
]

# Create the filtered dictionary
filtered_dummy_fetchResult = {
    "data": {
        "moldId": dummy_fetchResult['data']['moldId'],
        "counterId": dummy_fetchResult['data']['counterId'],
        "startHour": dummy_fetchResult['data']['startHour'],
        "contractedCycleTime": dummy_fetchResult['data']['contractedCycleTime'],
        "accelerations": [data['accelerations'] for data in filtered_data],
        "dataAccId": [data['dataAccId'] for data in filtered_data],
        "measurementDate": [data['measurementDate'] for data in filtered_data],
        "procChanged": [data['procChanged'] for data in filtered_data],
        "similarityMetric": [data['similarityMetric'] for data in filtered_data],
        "similarityMetricHr": [data['similarityMetricHr'] for data in filtered_data],
    }
}

# Assign the filtered dictionary back to dummy_fetchResult
dummy_fetchResult = filtered_dummy_fetchResult
## ... (previous code)

# Assign the filtered dictionary back to dummy_fetchResult
# ... (previous code)

# Print the filtered_dummy_fetchResult to check its content
print("Filtered Dummy Fetch Result:", filtered_dummy_fetchResult)

# Extract measurementDate values as strings
measurement_date = [str(ts) for ts in filtered_dummy_fetchResult['data']['measurementDate']]

# Join the measurementDate strings into a single string
measurement_date_str = ", ".join(measurement_date)

# Print the concatenated measurement dates
print("Measurement Dates:", measurement_date_str)


In [ ]:
from artifacts.utils import generate_hrs_template
print('Batch Start Hour: ',str(df['MEASUREMENT_DATE'][0])[:-4])
print(generate_hrs_template(
    str(2023042614),
    4,
    2,
))


In [ ]:
data = []

# Loop through the dictionary and extract relevant information
for key, value in ach.items():
    data.append({
        'Hour': value.hour,
        'Exists': value.exists,
        'SimilarityMetric': value.sim_metric_hr,
        'ProcChanged': value.proc_changed,
        'ToReturn': value.to_return,

        # Add more attributes as needed
    })

# Create a DataFrame from the collected data
df = pd.DataFrame(data)
df